# Final Project: Fake News Detection

By Felix Daubner - Hochschule der Medien

Module 'Supervised and Unsupervised Learning' - Prof. Dr.-Ing. Johannes Maucher

## Model Training

To-Do:
- Prepare data for training and validation
- Define model architecture

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

Feature Selection
- nur Text
- nur Kategorien (Channel, Person, etc.)
- alles

Performance vergleichen

- CNN?
- Neuronales Netz (MLP)
- LSTM?

This section contains the model training and feature selection. Different types of models should be trained and then compared to find out which model fits the challenge, to determine whether a political statement was fake-news or true, best. There are three types of models to be compared: MLP, CNN and LSTM. Those models should also vary in terms of hyperparameters like layers, neurons, optimization and else. The best model is evaluated and then optimized in the next section.

### Prepare and save data for training

In [2]:
data = pd.read_csv("data/processed.csv", sep=";", index_col=0)

Before starting defining the different models, the data is prepared for the training process. The neural network to be trained only takes numpy arrays as input. Thus, the data currently saved as a pandas DataFrame is converted in to a numpy array. In this conversion process, only "token", the encoded channel and issue columns and "truth" are kept meaning column 'statement' is dropped as it can not be used for model training.

In [7]:
X = np.array(data.drop(["statement"], axis=1).apply(np.array))
y = np.array(data["truth"])

The prepared data 'X' containing all the features and 'y' containing all targets to the corresponding features. Both arrays are saved as a csv-file.

In [ ]:
np.savetxt("data/X.csv", X, delimiter=";")
np.savetxt("data/y.csv", y, delimiter=";")